In [2]:
#IMPORT CELL
import sympy as sym
import numpy as np
from sympy import cos, sin, tan
import math
import matplotlib.pyplot as plt
from IPython.display import Markdown, display
# from google.colab import drive

Below are the help functions in previous homeworks, which you may need for this homework.

In [3]:

def integrate(f, xt, dt):
    """
    This function takes in an initial condition x(t) and a timestep dt,
    as well as a dynamical system f(x) that outputs a vector of the
    same dimension as x(t). It outputs a vector x(t+dt) at the future
    time step.

    Parameters
    ============
    dyn: Python function
        derivate of the system at a given step x(t),
        it can considered as \dot{x}(t) = func(x(t))
    xt: NumPy array
        current step x(t)
    dt:
        step size for integration

    Return
    ============
    new_xt:
        value of x(t+dt) integrated from x(t)
    """
    k1 = dt * f(xt)
    k2 = dt * f(xt+k1/2.)
    k3 = dt * f(xt+k2/2.)
    k4 = dt * f(xt+k3)
    new_xt = xt + (1/6.) * (k1+2.0*k2+2.0*k3+k4)
    return new_xt

def simulate(f, x0, tspan, dt, integrate):
    """
    This function takes in an initial condition x0, a timestep dt,
    a time span tspan consisting of a list [min_time, max_time],
    as well as a dynamical system f(x) that outputs a vector of the
    same dimension as x0. It outputs a full trajectory simulated
    over the time span of dimensions (xvec_size, time_vec_size).

    Parameters
    ============
    f: Python function
        derivate of the system at a given step x(t),
        it can considered as \dot{x}(t) = func(x(t))
    x0: NumPy array
        initial conditions
    tspan: Python list
        tspan = [min_time, max_time], it defines the start and end
        time of simulation
    dt:
        time step for numerical integration
    integrate: Python function
        numerical integration method used in this simulation

    Return
    ============
    x_traj:
        simulated trajectory of x(t) from t=0 to tf
    """
    N = int((max(tspan)-min(tspan))/dt)
    x = np.copy(x0)
    tvec = np.linspace(min(tspan),max(tspan),N)
    xtraj = np.zeros((len(x0),N))
    for i in range(N):
        xtraj[:,i]=integrate(f,x,dt)
        x = np.copy(xtraj[:,i])
    return xtraj

def animate_double_pend(theta_array,L1=1,L2=1,T=10):
    """
    Function to generate web-based animation of double-pendulum system

    Parameters:
    ================================================
    theta_array:
        trajectory of theta1 and theta2, should be a NumPy array with
        shape of (2,N)
    L1:
        length of the first pendulum
    L2:
        length of the second pendulum
    T:
        length/seconds of animation duration

    Returns: None
    """

    ################################
    # Imports required for animation.
    from plotly.offline import init_notebook_mode, iplot
    from IPython.display import display, HTML
    import plotly.graph_objects as go

    #######################
    # Browser configuration.
    def configure_plotly_browser_state():
        import IPython
        display(IPython.core.display.HTML('''
            <script src="/static/components/requirejs/require.js"></script>
            <script>
              requirejs.config({
                paths: {
                  base: '/static/base',
                  plotly: 'https://cdn.plot.ly/plotly-1.5.1.min.js?noext',
                },
              });
            </script>
            '''))
    configure_plotly_browser_state()
    init_notebook_mode(connected=False)

    ###############################################
    # Getting data from pendulum angle trajectories.
    xx1=L1*np.sin(theta_array[0])
    yy1=-L1*np.cos(theta_array[0])
    xx2=xx1+L2*np.sin(theta_array[0]+theta_array[1])
    yy2=yy1-L2*np.cos(theta_array[0]+theta_array[1])
    N = len(theta_array[0]) # Need this for specifying length of simulation

    ####################################
    # Using these to specify axis limits.
    xm=np.min(xx1)-0.5
    xM=np.max(xx1)+0.5
    ym=np.min(yy1)-2.5
    yM=np.max(yy1)+1.5

    ###########################
    # Defining data dictionary.
    # Trajectories are here.
    data=[dict(x=xx1, y=yy1,
               mode='lines', name='Arm',
               line=dict(width=2, color='blue')
              ),
          dict(x=xx1, y=yy1,
               mode='lines', name='Mass 1',
               line=dict(width=2, color='purple')
              ),
          dict(x=xx2, y=yy2,
               mode='lines', name='Mass 2',
               line=dict(width=2, color='green')
              ),
          dict(x=xx1, y=yy1,
               mode='markers', name='Pendulum 1 Traj',
               marker=dict(color="purple", size=2)
              ),
          dict(x=xx2, y=yy2,
               mode='markers', name='Pendulum 2 Traj',
               marker=dict(color="green", size=2)
              ),
        ]

    ################################
    # Preparing simulation layout.
    # Title and axis ranges are here.
    layout=dict(xaxis=dict(range=[xm, xM], autorange=False, zeroline=False,dtick=1),
                yaxis=dict(range=[ym, yM], autorange=False, zeroline=False,scaleanchor = "x",dtick=1),
                title='Double Pendulum Simulation',
                hovermode='closest',
                updatemenus= [{'type': 'buttons',
                               'buttons': [{'label': 'Play','method': 'animate',
                                            'args': [None, {'frame': {'duration': T, 'redraw': False}}]},
                                           {'args': [[None], {'frame': {'duration': T, 'redraw': False}, 'mode': 'immediate',
                                            'transition': {'duration': 0}}],'label': 'Pause','method': 'animate'}
                                          ]
                              }]
               )

    ########################################
    # Defining the frames of the simulation.
    # This is what draws the lines from
    # joint to joint of the pendulum.
    frames=[dict(data=[dict(x=[0,xx1[k],xx2[k]],
                            y=[0,yy1[k],yy2[k]],
                            mode='lines',
                            line=dict(color='red', width=3)
                            ),
                       go.Scatter(
                            x=[xx1[k]],
                            y=[yy1[k]],
                            mode="markers",
                            marker=dict(color="blue", size=12)),
                       go.Scatter(
                            x=[xx2[k]],
                            y=[yy2[k]],
                            mode="markers",
                            marker=dict(color="blue", size=12)),
                      ]) for k in range(N)]

    #######################################
    # Putting it all together and plotting.
    figure1=dict(data=data, layout=layout, frames=frames)
    iplot(figure1)

In [ ]:
# Srikanth's soln to problem 1
x, y = sym.symbols(r'x, y')
f = sin(x+y)*sin(x-y)

grad = [f.diff(x),f.diff(y)]
dgrad = [grad[0].diff(x), grad[1].diff(y)]

print('\033[1mThe gradient of f shown symbolically and substituting for x and y: ')
display(grad)
min_1 = [grad[0].subs({x:0, y:np.pi/2}),grad[1].subs({x:0, y:np.pi/2})]
display(min_1)
print('\033[1mThe second derivative of f shown symbolically and substituting for x and y: ')
display(dgrad)
dmin_1 = [dgrad[0].subs({x:0, y:np.pi/2}),dgrad[1].subs({x:0, y:np.pi/2})]
display(dmin_1)
print('\033[1mThis shows that the first order derivative is 0 at that point and the second order is positive,\ntherefore this proves the point is a local minimizer: ')


## Problem 2 (20pts)

In [ ]:
# Srikanth's soln to problem 2
L,m_1,m_2,k_1,k_2 = sym.symbols(r'L,m_1,m_2,k_1, k_2')

t = sym.symbols('t')
# define position as function of t
y1 = sym.Function(r'y_1')(t)
y2 = sym.Function(r'y_2')(t)
theta1 = sym.Function(r'\theta_1')(t)
theta2 = sym.Function(r'\theta_2')(t)
theta1dot = theta1.diff(t)
theta2dot = theta2.diff(t)

theta1ddot = theta1dot.diff(t)
theta2ddot = theta2dot.diff(t)

y1dot = y1.diff(t)
y2dot = y2.diff(t)

y1ddot = y1dot.diff(t)
y2ddot = y2dot.diff(t)

y1_sub = L*tan(theta1)
y2_sub = L*tan(theta2)

y1_subdot = y1_sub.diff(t)
y2_subdot = y2_sub.diff(t)

y1_subddot = y1_subdot.diff(t)
y2_subddot = y2_subdot.diff(t)

# Expression for kinetic energy
KE = 0.5*((m_1*y1dot**2) +(m_2*y2dot**2))
print("Expression for Kinetic Energy of the system y terms:")
display(KE)

# Expression for the potential energy
V = 0.5*((k_1*y1**2)+(k_2*(y2-y1)**2))
print("\nExpression for Potential Energy of the system y terms:")
display(V)

# Lagrangian is KE-V
L = KE - V
print("\n")
display(Markdown("Expression for Lagrangian of the system (y terms):"))
display(L)
display(Markdown("Expression for Lagrangian of the system (theta terms):"))
L_theta = L.subs({y1:y1_sub, y2:y2_sub, y1dot:y1_subdot, y2dot:y2_subdot})
display(L_theta)

dLdt1dot = L_theta.diff(theta1dot)
dLdt2dot = L_theta.diff(theta2dot)

EL_t1 = L_theta.diff(theta1) - dLdt1dot.diff(t)
EL_t2 = L_theta.diff(theta2) - dLdt2dot.diff(t)

# # theta1 = math.atan2(y1/L)
# # theta2 = math.atan(y2/L)
# display("**The equations for theta are:**")
# # display(theta1)

# EL_1 = ((-k_1*y1)-(k_2*(y1-y2)))-m_1*y1ddot
# print('Euler Lagrange for block 1 in terms of y: ')
# display(EL_1)

# EL_2 = (-k_2*(y2-y1))-m_2*y2ddot
# print('Euler lagrange for block 2 in terms of y: ')
# display(EL_2)

# EL_t1 = EL_1.subs({y1:y1_sub, y2:y2_sub, y1ddot:y1_subddot})
print('\033[1mEuler Lagrange for block 1 in terms of theta: ')
display(EL_t1)

# EL_t2 = EL_1.subs({y1:y1_sub, y2:y2_sub, y2ddot:y2_subddot})
print('\033[1mEuler lagrange for block 2 in terms of theta: ')
display(EL_t2)

# define left hand side as a matrix
lhs = sym.Matrix([EL_t1, EL_t2])

# define right hand side as a Matrix
rhs = sym.Matrix([0, 0])

# define the equations
print("The Euler Lagrange Equations set to 0")
eqn = sym.Eq(lhs, rhs)
display(eqn)

# solve it for both x and y
q = sym.Matrix([theta1ddot, theta2ddot])
soln = sym.solve(eqn, q, dict=True) # this will return the solution
                                     # as a Python dictionary
for sol in soln:
    print('\n\033[1mSolution: ')
    for v in q:
        display(sym.Eq(v, sol[v]))

In [ ]:
# Srikanth's Solution to problem 3

### Functions in terms of y as defined
y_1,y_2,m_1,m_2,k_1,k_2 = sym.symbols(r'y_1,y_2,m_1,m_2,k_1, k_2')
a_1 = ((-k_1*y_1)-(k_2*(y_1-y_2)))/m_1
print('\033[1mExpression for block 1 acceleration (y terms): ')
display(a_1)

a_2 = (-k_2*(y_2-y_1))/m_2
print('\033[1mExpression for block 2 acceleration (y terms): ')
display(a_2)

# NOTE: y2 = x1 + x2
t = sym.symbols('t')
# define position as function of t
x1 = sym.Function(r'x_1')(t)
x2 = sym.Function(r'x_2')(t)

# define velocity as derivative of position wrt t
x1dot = x1.diff(t)
x2dot = x2.diff(t)

# define acceleration as derivative of velocity wrt t
x1ddot = x1dot.diff(t)
x2ddot = x2dot.diff(t)

# Expression for kinetic energy
KE = 0.5*((m_1*x1dot**2) +(m_2*x2dot**2))
print("Expression for Kinetic Energy of the system:")
display(KE)

# Expression for the potential energy
V = 0.5*((k_1*x1**2)+(k_2*(x2-x1)**2))
print("\nExpression for Potential Energy of the system:")
display(V)

# Lagrangian is KE-V
L = KE - V
print("\n")
display(Markdown("**Expression for Lagrangian of the system:**"))
display(L)
q = sym.Matrix([x1, x2])
qdot = sym.Matrix([x1dot, x2dot])

# Lagrangian
print("The langrangian of the system")
display(sym.simplify(L))

# compute derivative wrt a vector, method 1
# wrap the expression into a SymPy Matrix
L_mat = sym.Matrix([L])
# SymPy Matrix has built-in method for Jacobian
dLdq = L_mat.jacobian(q)
dLdqdot = L_mat.jacobian(qdot)
ddtDL = dLdqdot.diff(t)

# Final equations:
EL = dLdq - ddtDL
# print('\n\033[1mDerivative of L w.r.t. q:')
# display(dLdq) # note that dJdq is 1-by-2 vector, not 2-by-1 as q
# print('\n\033[1mDerivative of L w.r.t. qdot:')
# display(dLdqdot) # note that dJdq is 1-by-2 vector, not 2-by-1 as q
# print('\n\033[1mDerivative of L w.r.t. qdot and t:')
# display(ddtDL) # note that dJdq is 1-by-2 vector, not 2-by-1 as q
# print("")
display(Markdown("**Final matrix of both halves of the equations combined**"))
display(sym.simplify(EL)) # note that dJdq is 1-by-2 vector, not 2-by-1 as q


## Problem 4 (10pts)



For the same double-pendulum system hanging in gravity in Homework 2 (shown above), take $q=[\theta_1, \theta_2]$ as the system configuration variables, with $R_1=R_2=1, m_1=m_2=1$. Symbolically compute the Hamiltonian of this system using Python's *SymPy* package.

---
**Turn in: Include the code used to symbolically compute the Hamiltonian of the system and the code output, which should the Hamiltonian of the system. Make sure you are using *SimPy*'s `.simplify()` functionality when printing your output.**

In [22]:
# Srikanth's Solution to problem 4

# t, g, m1, m2, R1, R2 = sym.symbols(r't, g, m_1, m_2, R_1, R_2')
t= sym.symbols(r't')

g = 9.8
R1 = 1 
R2 = 1
m1=1
m2 =1 
th1 = sym.Function(r'\theta_1')(t)
theta1 = th1 
th2 = sym.Function(r'\theta_2')(t)
theta2 = th2 
q = sym.Matrix([th1, th2])
qdot = q.diff(t)
qddot = qdot.diff(t)

theta1dot = qdot[0]
theta2dot = qdot[1]

theta1ddot = qddot[0]
theta2ddot = qddot[1]

p1x = R1*sym.sin(q[0])
p1y = -R1*sym.cos(q[0])
p2x = p1x + (R2*sym.sin(q[0]+q[1]))
p2y = p1y + (-R2*sym.cos(q[0]+q[1]))

p1xdot = p1x.diff(t)
p1ydot = p1y.diff(t)
p2xdot = p2x.diff(t)
p2ydot = p2y.diff(t)


display(theta1)
display(theta2)
display(theta1dot)
display(theta2dot)
display(theta1ddot)
display(theta2ddot)


m1_x = R1*sym.cos(theta1)
m1_y = R1*sym.sin(theta1)

# For m2
m2_x = R1*sym.cos(theta1) + R2*sym.cos(theta1+theta2)
m2_y = R1*sym.sin(theta1) + R2*sym.sin(theta1+theta2)

total_kinematic = 0.5 * m1 * (m1_x.diff(t)**2 + m1_y.diff(t)**2) + 0.5 * m2 * (m2_x.diff(t)**2 +
                                                                               m2_y.diff(t)**2)
total_potential = -m1*g*m1_x + -m2*g*m2_x

lagrangian = sym.simplify(total_kinematic - total_potential)



KE_4 = 0.5*m1*(p1xdot**2+p1ydot**2) + 0.5*m2*(p2xdot**2+p2ydot**2)
V_4 = m1*g*p1y + m2*g*p2y

# L_4 = KE_4 - V_4
# L_7 =sym.simplify(L_4)
# L_7 = sym.simplify(L_4.subs({g:9.8, R1:1, R2:1, m1:1, m2:1}))

L_7 = lagrangian


\theta_1(t)

\theta_2(t)

Derivative(\theta_1(t), t)

Derivative(\theta_2(t), t)

Derivative(\theta_1(t), (t, 2))

Derivative(\theta_2(t), (t, 2))

In [ ]:
# Srikanth's Solution to problem 5
H_no_g = H_sub.subs(g,9.8)
print("The hamiltonian")
display(H_no_g)
# Lagrangian
# print("The langrangian of the system")
# display(sym.simplify(L_4))

# compute derivative wrt a vector, method 1
# wrap the expression into a SymPy Matrix
L_mat = sym.Matrix([L_4])
# SymPy Matrix has built-in method for Jacobian
dLdq = L_mat.jacobian(q)
dLdqdot = L_mat.jacobian(qdot)
ddtDL = dLdqdot.diff(t)

# Final equations:
EL = dLdq - ddtDL

# define left hand side as a matrix
lhs = sym.Matrix([EL[0], EL[1]])

# define right hand side as a Matrix
rhs = sym.Matrix([0, 0])

# define the equations
# print("The Euler Lagrange Equations set to 0:")
eqn = sym.Eq(sym.simplify(lhs), rhs)
# display(eqn)

# solve it for both x and y
q2 = sym.Matrix([theta1ddot, theta2ddot])
soln = sym.solve(eqn, q2, dict=True) # this will return the solution

# display(soln)


func_1 = sym.lambdify([m_1,m_2,R1,R2,g,theta1,theta2,theta1dot,theta2dot ], soln[0][theta1ddot])

func_2 = sym.lambdify([m_1,m_2,R1,R2,g,theta1,theta2,theta1dot,theta2dot ], soln[0][theta2ddot])

def xddot(theta1, theta2, theta1dot, theta2dot):
    """
    Acceleration of the particle in terms of
    position and velocity. Here it's a constant.
    """
    theta1_acc = func_1(1, 1, 1, 1, 9.8, theta1, theta2, theta1dot, theta2dot)
    theta2_acc = func_2(1, 1, 1, 1, 9.8, theta1, theta2, theta1dot, theta2dot)
    return theta1_acc, theta2_acc

def dyn(s):
    """
    System dynamics function (extended)

    Parameters
    ============
    s: NumPy array
        s = [x, xdot] is the extended system
        state vector, includng the position and
        the velocity of the particle

    Return
    ============
    sdot: NumPy array
        time derivative of input state vector,
        sdot = [xdot, xddot]
    """
    return np.array([s[2], s[3], *xddot(s[0], s[1], s[2], s[3])])

s0 = np.array([-np.pi/2, -np.pi/2, 0, 0])
tspan = [0,10]
trajectory = simulate(dyn, s0, tspan, 0.01, integrate)
tvec = np.linspace(min(tspan),max(tspan), 1000)
H_lamb = sym.lambdify([th1,th2,qdot[0],qdot[1]], H_no_g)
H_list = []

for i in range(0,1000):
  Hi = H_lamb(trajectory[0][i],trajectory[1][i],trajectory[2][i],trajectory[3][i])
  H_list.append(Hi)

fig, ax = plt.subplots()
print('\033[1mThe Hamiltonian plotted over time')
plt.plot(tvec, H_list)

In [ ]:
def euler_integrate(diff_fcn, x_0, delta_t):
  x_1 = x_0 + diff_fcn(x_0)*delta_t
  return x_1

s02 = np.array([-np.pi/2, -np.pi/2, 0, 0])
tspan2 = [0,10]
trajectory2 = simulate(dyn, s02, tspan2, 0.01, euler_integrate)
tvec2 = np.linspace(min(tspan2),max(tspan2), 1000)
H_lamb2 = sym.lambdify([th1,th2,qdot[0],qdot[1]], H_no_g)
H_list2 = []

for i in range(0,1000):
  Hi2 = H_lamb2(trajectory2[0][i],trajectory2[1][i],trajectory2[2][i],trajectory2[3][i])
  H_list2.append(Hi2)

fig, ax = plt.subplots()
print('\033[1mThe Hamiltonian plotted over time')
plt.plot(tvec2, H_list2)

print("\033[1mClearly based on the graph above, there is a clear divergence \nwith the Euler methon where RK4 was relatively stable")

## Problem 7 (20pts)

For the same double-pendulum you simulated in Problem 4 with same parameters and initial condition, now add a constraint to the system such that the distance between the second pendulum and the origin is fixed at $\sqrt{2}$. Simulate the system with same parameters and initial condition, and animate the system with the same animate function provided in Homework 2.

> *Hint 1: What do you think the equations of motion should look like? Think about how the system will behave after adding the constraint. With no double, you can solve this problem using $\phi$ and all the following results for constrained Euler-Lagrange equations, however, if you really understand this constrained system, things might be much easier, and you can actually treat it as an unconstrained system.*

---
**Turn in: Include the code used to numerically evaluate, simulate and animate the system. Also, upload the video of animation separately through Canvas in ".mp4" format. You can use screen capture or record the screen directly with your phone.**

In [31]:
# Srikanth's solution to problem 7
lamb7 = sym.symbols('\lambda')
phi = p2x**2 + p2y**2 - 2

phiddot = sym.simplify(phi.diff(t).diff(t))

display(phiddot)

L_mat7 = sym.Matrix([L_7])
# SymPy Matrix has built-in method for Jacobian
dLdq7 = L_mat7.jacobian(q).T
dLdqdot7 = L_mat7.jacobian(qdot).T
ddtDL7 = dLdqdot7.diff(t)

# Final equations:
EL7 = ddtDL7 - dLdq7 

# Redefine lhs as matrix to get constraints
grad1 = sym.simplify(phi.diff(th1)*lamb7)
grad2 = sym.simplify(phi.diff(th2)*lamb7)
lhs7 = sym.Matrix([sym.simplify(EL7[0]-grad1), sym.simplify(EL7[1]-grad2), phiddot])
display(grad1)
display(grad2)

display("=========")
# define right hand side as a Matrix
rhs7 = sym.Matrix([0, 0, 0])

# print("The Euler Lagrange Equations set to 0:")
eqn7 = sym.Eq(sym.simplify(lhs7), rhs7)
# display(eqn)

# solve it for both x and y
q7 = [theta1ddot, theta2ddot, lamb7]
soln7 = sym.solve(eqn7, q7, dict=True) # this will return the solution

func_17 = sym.lambdify([th1,th2,theta1dot,theta2dot], soln7[0][theta1ddot])
display(theta1ddot)
display(theta2ddot)
func_27 = sym.lambdify([th1,th2,theta1dot,theta2dot ], soln7[0][theta2ddot])
def xddot7(theta1, theta2, theta1dot, theta2dot):
    """
    Acceleration of the particle in terms of
    position and velocity. Here it's a constant.
    """
    theta1_acc = func_17(theta1, theta2, theta1dot, theta2dot)
    theta2_acc = func_27(theta1, theta2, theta1dot, theta2dot)
    return theta1_acc, theta2_acc

def dyn7(s):
    """
    System dynamics function (extended)

    Parameters
    ============
    s: NumPy array
        s = [x, xdot] is the extended system
        state vector, includng the position and
        the velocity of the particle

    Return
    ============
    sdot: NumPy array
        time derivative of input state vector,
        sdot = [xdot, xddot]
    """

    # return np.array([s[2], s[3], *xddot7(s[0], s[1], s[2], s[3])])
    acc1 = func_17(s[0], s[1], s[2], s[3])
    acc2 = func_27(s[0], s[1], s[2], s[3])
    return np.array([s[2], s[3], acc1 , acc2      ])

s7 = np.array([-np.pi/2, -np.pi/2, 0, 0]) # initial conditions
# simulat from t=0 to 5, since dt=0.1, the returned trajectory
# will have 10/0.1=100 time steps, each time step contains extended
# system state vector [theta1(t), theta2(2), theta1dot(t), theta2dot(t)]
traj7 = simulate(dyn7, s7, [0, 10], 0.01, integrate)
display(traj7)
print('\033[1mShape of traj: \033[0m', traj7.shape)


tspan7 = [0,10]
# fig, ax = plt.subplots()
# tvec = np.linspace(min(tspan),max(tspan), 500)
# ax.plot(tvec, traj[0])
# ax.plot(tvec, traj[1])
# plt.plot(traj[0])
# plt.plot(traj[1])

# display(Markdown("**Animation Below**"))
# animate_double_pend(traj7[0:2,:], T=10)

-2*sin(\theta_2(t))*Derivative(\theta_2(t), (t, 2)) - 2*cos(\theta_2(t))*Derivative(\theta_2(t), t)**2

-\lambda*(9.8*sin(\theta_1(t) + \theta_2(t)) + 19.6*sin(\theta_1(t)))

-\lambda*(9.8*sin(\theta_1(t) + \theta_2(t)) + 1.0*sin(\theta_2(t))*Derivative(\theta_1(t), t)**2 + 1.0*sin(\theta_2(t))*Derivative(\theta_1(t), t)*Derivative(\theta_2(t), t))

'========='

Derivative(\theta_1(t), (t, 2))

Derivative(\theta_2(t), (t, 2))

array([[-1.57079633, -1.57079633, -1.57079633, ..., -1.57079633,
        -1.57079633, -1.57079633],
       [-1.57079633, -1.57079633, -1.57079633, ..., -1.57079633,
        -1.57079633, -1.57079633],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

Shape of traj:  (4, 1000)


## Problem 8 (5pts)

For the same system with same constraint in Problem 6, simulate the system with initial condition $\theta_1=\theta_2=-\frac{\pi}{4}$, which actually violates the constraint! Simulate the system and see what happen, what do you think is the actual influence after adding this constraint?

---
**Turn in: Your thoughts about the actual effect of the constraint in this system. Note that you don't need to include any code for this problem.**

In [ ]:
print("The system can never get to the constraint since the pendulum is fixed so the lambda value will never reach 0. It will always oscillate with a positive absolute value")